In [2]:
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', 50)

In [3]:
def load_data(path):
    data = pd.read_csv(path)
    
    return data

def prep_data(reco_group_df, prod_df, cw_df):
    # merge data
    merge_data = pd.merge(reco_group_df, prod_df, on=['prod_id', 'brad_date', 'ch_no', 'ch_nm'], how='left')
    merge_data = pd.merge(merge_data, cw_df, on='cw_id', how='left')
    
    return merge_data

def run_data_func(data_path1, data_path2, data_path3):
    # 홈쇼핑 추천 컨테이너 필요 요소
    # 카피라이팅, 채널번호, 채널이름, 상품 이미지, 상품제목, 상품가격
    
    # load data
    reco_group_df = load_data(data_path1)
    prod_df = load_data(data_path2)
    cw_df = load_data(data_path3)
    
    # prep_data
    merge_data = prep_data(reco_group_df, prod_df, cw_df)
    
    return merge_data

In [4]:
# data test
data_path = 'data/'
reco_group_df_path = data_path + 'ext_reco_group_df.dat'
prod_df_path = data_path + 'ext_prod_df.dat'
cw_df_path = data_path + 'ext_cw_df.dat'

merge_data = run_data_func(reco_group_df_path, prod_df_path, cw_df_path)
print('merge_data: ', merge_data.shape)
merge_data.columns

merge_data:  (186, 30)


Index(['sa_id', 'p_id', 'brad_date', 'ch_svc_id', 'ch_no', 'ch_nm', 'prod_id',
       'cw_id', 'bom_st_dt', 'bom_fns_dt', 'bom_prod_nm', '대분류', '중분류', '소분류',
       '브랜드', '세분류', 'productImgUrl', 'mUrl', 'originPrice', 'price',
       'cw_date', '선호그룹', '메타그룹', '카피라이팅', '평균', '매력성', '자연스러움', '적합성', '길이',
       '종결어미'],
      dtype='object')

In [5]:
# 3자리마다 콤마 넣기
merge_data['price'] = [format(x, ',') for x in merge_data['price']]

# 채널 번호 000으로 바꾸기 (7번 -> 007번)
merge_data['ch_no'] = [format(x, '03') for x in merge_data['ch_no']]
merge_data['ch_no'] = merge_data['ch_no'].astype(str)

In [6]:
# s급 채널 위로?


In [6]:
# merge_data['disc_rate'] = np.round((merge_data['originPrice'] - merge_data['price']) / merge_data['originPrice'] * 100, 0)
# merge_data.head()

In [12]:
# 데이터 수정

# # 3자리마다 콤마 넣기
# merge_data['price'] = [format(x, ',') for x in merge_data['price']]

# # 채널 번호 000으로 바꾸기 (7번 -> 007번)
# merge_data['ch_no'] = [format(x, '03') for x in merge_data['ch_no']]
# merge_data['ch_no'] = merge_data['ch_no'].astype(str)

# 할인율 만들기


merge_data.head(1)

,sa_id,p_id,brad_date,ch_svc_id,ch_no,ch_nm,prod_id,cw_id,bom_st_dt,bom_fns_dt,bom_prod_nm,대분류,중분류,소분류,브랜드,세분류,productImgUrl,mUrl,originPrice,price,cw_date,선호그룹,메타그룹,카피라이팅,평균,매력성,자연스러움,적합성,길이,종결어미
0,0,0,2024-02-04,395,12,KT알파 쇼핑,3951707042540,4016000,2024-02-04 10:29:00,2024-02-04 11:28:58,"특사이즈 총 24인분! [해화당] 이영자의 뼈없는 갈비탕 900g x 8팩, 총 7...",식품/건강,축산물,축산가공식품,해화당,뼈없는 갈비탕,http://imgs.kshop.co.kr//d2/product/emc/202312...,http://m.kshop.co.kr/display/mc/product/263484...,63900,54320,2024-02-01,삼시세끼,식품,"간편하게 요리, 겨울밤을 닮은 삼시세끼",8.75,9,8,8,10,P


In [61]:
# merge_data.to_csv(data_path + 'merge_df.dat', index=False, )

In [62]:
merge_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186 entries, 0 to 185
Data columns (total 30 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   sa_id          186 non-null    int64  
 1   p_id           186 non-null    int64  
 2   brad_date      186 non-null    object 
 3   ch_svc_id      186 non-null    int64  
 4   ch_no          186 non-null    object 
 5   ch_nm          186 non-null    object 
 6   prod_id        186 non-null    int64  
 7   cw_id          186 non-null    int64  
 8   bom_st_dt      186 non-null    object 
 9   bom_fns_dt     186 non-null    object 
 10  bom_prod_nm    186 non-null    object 
 11  대분류            186 non-null    object 
 12  중분류            186 non-null    object 
 13  소분류            186 non-null    object 
 14  브랜드            186 non-null    object 
 15  세분류            186 non-null    object 
 16  productImgUrl  186 non-null    object 
 17  mUrl           186 non-null    object 
 18  originPric

In [7]:
user1 = merge_data['p_id'].unique()[1]
user_df = merge_data[merge_data['p_id'] == user1]
mod_columns = ['sa_id', 'p_id', 'brad_date', 'ch_svc_id', 'ch_no', 'ch_nm', 'bom_st_dt', 'bom_fns_dt', 'bom_prod_nm',
               'prod_id', 'price', 'originPrice', 'productImgUrl', 'mUrl']
user_df[mod_columns].to_dict('records')

[{'sa_id': 1,
  'p_id': 1,
  'brad_date': '2024-02-03',
  'ch_svc_id': 362,
  'ch_no': '014',
  'ch_nm': '홈&쇼핑',
  'bom_st_dt': '2024-02-03 19:30:00',
  'bom_fns_dt': '2024-02-03 20:29:58',
  'bom_prod_nm': '(20% 세일)[유현수] 블랙앵거스 LA갈비 400g x 7팩',
  'prod_id': 3621706988600,
  'price': '69,900',
  'originPrice': 87900,
  'productImgUrl': 'http://image.hnsmall.com/images/goods/422/41504422_g.jpg',
  'mUrl': 'http://m.hnsmall.com/goods/view/41504422?channel_code=21363&utm_source=OllehPortal&utm_medium=AF&utm_campaign=2CH_None_41504422'},
 {'sa_id': 1,
  'p_id': 1,
  'brad_date': '2024-02-01',
  'ch_svc_id': 478,
  'ch_no': '017',
  'ch_nm': 'SK 스토아',
  'bom_st_dt': '2024-02-01 15:41:00',
  'bom_fns_dt': '2024-02-01 16:40:58',
  'bom_prod_nm': '[제주어부] 300g이상 6마리 24토막',
  'prod_id': 4781706802060,
  'price': '58,900',
  'originPrice': 59900,
  'productImgUrl': 'http://imagecdn.skstoa.com/goods/212/35749212_g.jpg',
  'mUrl': 'http://m.skstoa.com/display/goods/35749212?orderMedia=62&mediaCode=E

In [64]:
pd.read_csv('data/merge_df.dat')

,sa_id,p_id,brad_date,ch_svc_id,ch_no,ch_nm,prod_id,cw_id,bom_st_dt,bom_fns_dt,...,cw_date,선호그룹,메타그룹,카피라이팅,평균,매력성,자연스러움,적합성,길이,종결어미
0,0,0,2024-02-04,395,12,KT알파 쇼핑,3951707042540,4016000,2024-02-04 10:29:00,2024-02-04 11:28:58,...,2024-02-01,삼시세끼,식품,"간편하게 요리, 겨울밤을 닮은 삼시세끼",8.75,9,8,8,10,P
1,0,0,2024-02-02,395,12,KT알파 쇼핑,3951706838120,4016000,2024-02-02 01:42:00,2024-02-02 02:41:58,...,2024-02-01,삼시세끼,식품,"간편하게 요리, 겨울밤을 닮은 삼시세끼",8.75,9,8,8,10,P
2,0,0,2024-02-02,362,14,홈&쇼핑,3621706839200,4016000,2024-02-02 02:00:00,2024-02-02 02:59:58,...,2024-02-01,삼시세끼,식품,"간편하게 요리, 겨울밤을 닮은 삼시세끼",8.75,9,8,8,10,P
3,0,0,2024-02-04,478,17,SK 스토아,4781707054060,4016000,2024-02-04 13:41:00,2024-02-04 14:40:58,...,2024-02-01,삼시세끼,식품,"간편하게 요리, 겨울밤을 닮은 삼시세끼",8.75,9,8,8,10,P
4,0,0,2024-02-01,308,2,NS홈쇼핑,3081706798100,4016000,2024-02-01 14:35:00,2024-02-01 15:24:58,...,2024-02-01,삼시세끼,식품,"간편하게 요리, 겨울밤을 닮은 삼시세끼",8.75,9,8,8,10,P
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181,9,9,2024-02-02,323,4,롯데홈쇼핑,3231706887800,4026000,2024-02-02 15:30:00,2024-02-02 16:29:58,...,2024-02-01,건강관리,건강관리,"고급 건강관리, 포근한 웰빙 시작",8.75,8,8,9,10,P
182,9,9,2024-02-01,323,4,롯데홈쇼핑,3231706812200,4026000,2024-02-01 18:30:00,2024-02-01 19:33:58,...,2024-02-01,건강관리,건강관리,"고급 건강관리, 포근한 웰빙 시작",8.75,8,8,9,10,P
183,9,9,2024-02-03,270,6,CJ ONSTYLE,2701706944800,4026000,2024-02-03 07:20:00,2024-02-03 08:19:58,...,2024-02-01,건강관리,건강관리,"고급 건강관리, 포근한 웰빙 시작",8.75,8,8,9,10,P
184,9,9,2024-02-01,270,6,CJ ONSTYLE,2701706749200,4026000,2024-02-01 01:00:00,2024-02-01 01:59:58,...,2024-02-01,건강관리,건강관리,"고급 건강관리, 포근한 웰빙 시작",8.75,8,8,9,10,P


In [15]:
merge_data[merge_data['bom_prod_nm'].str.contains('임성근')]['productImgUrl'][60]

'http://img.shoppingntmall.com/goods/463/14171463_h.jpg'

In [10]:
merge_data['productImgUrl'][0]

'http://imgs.kshop.co.kr//d2/product/emc/202312/26/34/847/2634847_550_20231221155612.jpg'

In [8]:
merge_data.head(1)

,sa_id,p_id,brad_date,ch_svc_id,ch_no,ch_nm,prod_id,cw_id,bom_st_dt,bom_fns_dt,...,cw_date,선호그룹,메타그룹,카피라이팅,평균,매력성,자연스러움,적합성,길이,종결어미
0,0,0,2024-02-04,395,12,KT알파 쇼핑,3951707042540,4016000,2024-02-04 10:29:00,2024-02-04 11:28:58,...,2024-02-01,삼시세끼,식품,"간편하게 요리, 겨울밤을 닮은 삼시세끼",8.75,9,8,8,10,P
